In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from datetime import date
from bs4 import BeautifulSoup
import sys
import time
import pandas as pd
import datetime

now = datetime.datetime.now().strftime("%Y%m%d")

In [2]:
def listToString(s): 
    str1 = "" 
    for ele in s: 
        str1 += ele  
    return str1 

In [3]:
#new chrome driver
def OpenChromeDriver():
    #開啟瀏覽器新分頁，下載google chrome driver
    option = webdriver.ChromeOptions()

    #fix chrome 103 problem
    option.binary_location = "/Applications/Google Chrome Beta.app/Contents/MacOS/Google Chrome Beta"

    prefs={"profile.default_content_setting_values.notifications" : 2}
    option.add_experimental_option("prefs",prefs)
    gchrome=webdriver.Chrome(options=option)
    return gchrome

def FinishedCrawler(driver):
    driver.close()

In [4]:
#抓取幾頁
def getPageNum(url,driver):
    driver.get(url)
            
    #Get Comment       
    soup = BeautifulSoup(driver.page_source,"html.parser")
    pages = soup.find("div", class_="article_multi_page clear")
    try:
        page_num = len(pages.find_all("a"))
    except:
        return 1
    else:
        return page_num 

In [5]:
#抓取貼文連結
def getPostLink(url,driver):
    driver.get(url)
    
    try:
        for i in range(0,5):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            i += 1
            time.sleep(2)
    except OSError:
        time.sleep(15)
        
        for i in range(0,5):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            i += 1
            time.sleep(2)   
    post_link_list = []
    soup = BeautifulSoup(driver.page_source,"html.parser")
    for l in soup.find_all("div", class_="content_col"):
        temp_soup = BeautifulSoup(str(l),"html.parser")
        temp_target = temp_soup.find("a")
        temp = []
        temp.append(temp_target.get_text().strip().split('\t'))
        temp.append("https://www.dcfever.com"+temp_target["href"])
        post_link_list.append(temp)
    link_df = pd.DataFrame(data=post_link_list, columns=['Post Title', 'Link'])
    return link_df

In [6]:
#取得貼文日期

def getPostDate(page_soup):
    date_tag_string = page_soup.find("div", class_="article_date").get_text()
    date_tag_string = date_tag_string[date_tag_string.find("發表於")+3:date_tag_string.find("日")]
    year = date_tag_string[:date_tag_string.find("年")].strip()
    month = date_tag_string[date_tag_string.find("年")+1:date_tag_string.find("月")].strip()
    day = date_tag_string[date_tag_string.find("月")+1:].strip()
    post_date = datetime.date(int(year),int(month),int(day))
    return post_date.strftime("%Y/%m/%d")


In [7]:
#抓取貼文留言
def getPostContent(first_url,driver):
    
    urls=[]
    content_all=[]
    post_date = ""
    
    page_num = getPageNum(first_url,driver)
    if (page_num>1):
        for i in range(1,page_num+1):
            urls.append(first_url+"&page="+str(i))
        for url in urls:
            driver.get(url)
        #Get Comment       
            soup = BeautifulSoup(driver.page_source,"html.parser")
            if (url.find("&page=1") != -1):
                post_date = getPostDate(soup)
        
        content_all+=soup.find("div",class_="article_content").get_text(strip=True)
    
    else:
        driver.get(first_url)
        soup = BeautifulSoup(driver.page_source,"html.parser")
        post_date = getPostDate(soup)
        content_all+=soup.find("div",class_="article_content").get_text(strip=True)

    FinishedCrawler(driver)
    return listToString(content_all), post_date

#option, adding getdata

import os
phone_df = pd.read_csv("../crawler_data/Phone_Data_20220809.csv")
get_data_phone_df = pd.DataFrame(data=[], columns=['Brand_Name', 'Phone_Name', 'Link', 'Icon'])
path = "../crawler_data/"
dirs = os.listdir( path )
for brand_dir in dirs:
    if(brand_dir != ".DS_Store" and brand_dir != "較舊" and brand_dir.find("csv")==-1):
        temp_brand_dir = os.listdir( path+brand_dir+"/Link" )
        for phone in temp_brand_dir:
            temp_phone_row = phone_df.loc[phone_df['Phone_Name'] == phone]
            get_data_phone_df = pd.concat([get_data_phone_df, temp_phone_row], axis=0)

file_name = "../crawler_data/Get_Phone_Data_"+now+".csv"
#get_data_phone_df.to_csv(file_name, index=False, encoding="utf-8")

In [9]:
get_data_phone_df = pd.read_csv("../crawler_data/Get_Phone_Data_20220816.csv")

In [14]:
dcfever_posts_df = pd.DataFrame(data=[], columns=['Post Title', 'Link'])
phone_name = []
for index, row in get_data_phone_df.iterrows():
    try:
        driver = OpenChromeDriver()
        report_link = row["Link"].replace("specification","articles")
        temp_phone_link_df = getPostLink(report_link, driver)
        dcfever_posts_df = pd.concat([dcfever_posts_df, temp_phone_link_df], axis=0)
        for i in range(len(temp_phone_link_df)):
            phone_name.append(row["Phone_Name"])
        time.sleep(0.5)
    except:
        print(row["Phone_Name"])
    finally:
        FinishedCrawler(driver)

dcfever_posts_df["Phone_Name"] = phone_name
dcfever_posts_df


,Post Title,Link,Phone_Name
0,[iPhone SE 3 詳測：大量引誘吸 Android 客],https://www.dcfever.com/news/readnews.php?id=3...,iPhone SE 3
1,[iPhone SE 3 開箱：配件有一些轉變],https://www.dcfever.com/news/readnews.php?id=3...,iPhone SE 3
2,[iPhone SE 3 親民價發表：5G、A15 SoC 外還有不少強化],https://www.dcfever.com/news/readnews.php?id=3...,iPhone SE 3
0,[【iOS 15.4 新功能】iPhone 13 Pro 戴口罩使用 Face ID 實試],https://www.dcfever.com/news/readnews.php?id=3...,iPhone 13
1,[iPhone iOS 15.1 更新：新增 ProRes 影片及 FaceTime 同播同享],https://www.dcfever.com/news/readnews.php?id=3...,iPhone 13
...,...,...,...
8,[【攝力分析】Sony Xperia Pro-I 終於有機取代一般 DC？],https://www.dcfever.com/news/readnews.php?id=3...,Sony Xperia Pro-I
9,"[Sony Xperia Pro-I 實試：1"" Sensor 手機威力先睹為快]",https://www.dcfever.com/news/readnews.php?id=3...,Sony Xperia Pro-I
10,"[Sony Xperia Pro-I 發表香港有價：不一樣的 1"" Sensor、首發 4K...",https://www.dcfever.com/news/readnews.php?id=3...,Sony Xperia Pro-I
0,[vivo V23 5G 測試：5000 萬像素前鏡自拍多玩法],https://www.dcfever.com/news/readnews.php?id=3...,vivo V23 5G


In [15]:
file_name = "../crawler_data/Temp_DcFever_Phone_Data_"+now+".csv"
dcfever_posts_df.to_csv(file_name, index=False, encoding="utf-8")